In [1]:
import pandas as pd


In [2]:
df=pd.read_csv('glassdoor_jobs.csv')

In [4]:
df.head(10)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
5,5,Data Scientist,$71K-$119K (Glassdoor est.),CyrusOne is seeking a talented Data Scientist ...,3.4,CyrusOne\n3.4,"Dallas, TX","Dallas, TX",201 to 500 employees,2000,Company - Public,Real Estate,Real Estate,$1 to $2 billion (USD),"Digital Realty, CoreSite, Equinix"
6,6,Data Scientist,$54K-$93K (Glassdoor est.),Job Description\n\n**Please only local candida...,4.1,ClearOne Advantage\n4.1,"Baltimore, MD","Baltimore, MD",501 to 1000 employees,2008,Company - Private,Banks & Credit Unions,Finance,Unknown / Non-Applicable,-1
7,7,Data Scientist,$86K-$142K (Glassdoor est.),Advanced Analytics – Lead Data Scientist\nOver...,3.8,Logic20/20\n3.8,"San Jose, CA","Seattle, WA",201 to 500 employees,2005,Company - Private,Consulting,Business Services,$25 to $50 million (USD),-1
8,8,Research Scientist,$38K-$84K (Glassdoor est.),SUMMARY\n\nThe Research Scientist I will be ta...,3.3,Rochester Regional Health\n3.3,"Rochester, NY","Rochester, NY",10000+ employees,2014,Hospital,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD),-1
9,9,Data Scientist,$120K-$160K (Glassdoor est.),isn’t your usual company. Our work is powered ...,4.6,<intent>\n4.6,"New York, NY","New York, NY",51 to 200 employees,2009,Company - Private,Internet,Information Technology,$100 to $500 million (USD),"Clicktripz, SmarterTravel"


# *salary parsing*


In [5]:



df=df[df['Salary Estimate']!= '-1']

In [6]:
salary=df['Salary Estimate'].apply(lambda x:x.split('(')[0])

In [7]:
#remove K's & Dollar sign
minus_KDs=salary.apply(lambda x:x.replace('K',' ').replace('$',' '))

In [8]:
# all salary cases with 'per hour in them'
df['hourly']=df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

In [12]:
df['hourly'].sum()

24

In [13]:
#all cases with 'employer provided' in them 
df['employer provided']=df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

In [16]:
min_hr=minus_KDs.apply(lambda x:x.lower().replace('per hour'," ").replace('employer provided salary:'," "))

In [18]:
df['min_salary']=min_hr.apply(lambda x:int(x.split('-')[0]))
df['max_salary']=min_hr.apply(lambda x:int(x.split('-')[1]))

In [23]:
df['avg_salary']=(df.min_salary+df.max_salary)/2

# *company name text only*

In [25]:
df['company_txt']=df.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'][:-3],axis=1)

# *state field*


In [26]:
df['job_state']=df.Location.apply(lambda x: x.split(',')[1])

In [27]:
df['same_state']=df.apply(lambda x: 1 if x.Location==x.Headquarters else 0,axis =1)

#  *company age*

In [28]:
df['company_age']=df.Founded.apply(lambda x: x if x<1 else 2023-x)


# *parsing of job description*

In [29]:
df['python']=df['Job Description'].apply(lambda x: 1 if 'python'in x.lower() else 0)
df['r-studio']=df['Job Description'].apply(lambda x: 1 if 'r-studio'in x.lower() else 0)
df['spark']=df['Job Description'].apply(lambda x: 1 if 'spark'in x.lower() else 0)
df['aws']=df['Job Description'].apply(lambda x: 1 if 'aws'in x.lower() else 0)
df['excel']=df['Job Description'].apply(lambda x: 1 if 'excel'in x.lower() else 0)

In [32]:
df.excel.value_counts()

1    388
0    354
Name: excel, dtype: int64

In [33]:
df.python.value_counts()

1    392
0    350
Name: python, dtype: int64

In [35]:
df['r-studio'].value_counts()

0    741
1      1
Name: r-studio, dtype: int64

In [36]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'company_age', 'python',
       'r-studio', 'spark', 'aws', 'excel'],
      dtype='object')

In [38]:
df_output=df.drop('Unnamed: 0',axis=1)

In [39]:
df_output.to_csv('Cleaned_salary_data.csv',index=False)